# Query Six A Preprocessing

#### Are there any listings that a reviewer has reviewed more than thrice that is also available in the same month as was reviewed by them previously? (check against all the months that the previous reviews were posted on, if any match then it qualifies)

#### Goal: Clean the data files in prepartion for loading into Cassandra on Azure

In [43]:
#Imports
import numpy as np
import pandas as pd

In [44]:
#Uploading the data
pdx_reviews = pd.read_csv('PDX_reviews.csv')
la_reviews = pd.read_csv('la_reviews.csv')
sd_reviews = pd.read_csv('sd_reviews.csv')
salem_reviews = pd.read_csv('salem_reviews.csv')

In [45]:
salem_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,199568,486392,2011-08-29,66482,Victoria & Rob,"The Garden Apartment is lovely. It is open, ai..."
1,199568,491741,2011-08-31,66482,Victoria & Rob,We were so pleased with the apartment and our ...
2,199568,608314,2011-10-08,201283,Jenny,I stayed at Sara's apt.for two weeks while wo...
3,199568,615016,2011-10-10,66482,Victoria & Rob,My aunt and I enjoyed our stay in the lovely g...
4,199568,644195,2011-10-18,933601,Linda And Jerry,We truly enjoyed a restful time and environmen...


In [46]:
#Keeping only the columns we need (listing_id, id, date, reviewer_id, reviewer_name (for 6b))
pdx_select = pdx_reviews[["listing_id", "id","date","reviewer_id", "reviewer_name"]]
la_select = la_reviews[["listing_id", "id","date","reviewer_id", "reviewer_name"]]
sd_select = sd_reviews[["listing_id", "id","date","reviewer_id", "reviewer_name"]]
salem_select = salem_reviews[["listing_id", "id","date","reviewer_id", "reviewer_name"]]

In [47]:
#Combining all reviews for all cities
df_list = [pdx_select, la_select, sd_select, salem_select]
all_reviews = pd.concat(df_list)

#Testing concat
indiv_lengths = len(pdx_select) + len(la_select) + len(sd_select) + len(salem_select)
if len(all_reviews) == indiv_lengths:
    print("Concat worked correctly. New len: "+str(len(all_reviews)))
else:
    print("Concat did not merge right")

Concat worked correctly. New len: 2573162


In [48]:
#Converting date to datetime and adding month column
all_reviews['date'] = pd.to_datetime(all_reviews['date'])
all_reviews['review_month'] = all_reviews['date'].dt.month

In [49]:
#Removing date column
final_reviews = all_reviews.loc[:, all_reviews.columns!='date']
final_reviews

,listing_id,id,reviewer_id,reviewer_name,review_month
0,12899,24767,69327,Stuart,1
1,12899,29230,72846,John,3
2,12899,29806,84196,Lois,3
3,12899,32572,89114,Troy,3
4,12899,32862,100318,Cathy,4
...,...,...,...,...,...
14881,806288048006012567,840008042809282936,219002106,Cheryl,3
14882,806288048006012567,840728285056856719,52054229,Sarah,3
14883,808194120211324072,822551660654988407,82164517,Jody,2
14884,810859442087199945,818218402306004794,415177874,Alanmichael,2


In [10]:
'''#Saving the final reviews table for the creation of the dummy availability table
final_reviews.to_csv("q6_comb_reviews.csv")'''

### Cleaning the Availability Table
Steps:
    1. Load table
    2. Change date to a datetime col
    3. Create a avail_month col
    4. Remove all columns which aren't listing_id, avail_month, and true_availability
    5. Combine the avail_month rows into a single col per listing id with comma separated values
    6. Remove the true_availability = False column
    7. Save dataset'''

In [105]:
#Loading the availability table
avail_table = pd.read_csv('final_avail_data.csv')
avail_table.head()

,Unnamed: 0,listing_id,date,available,minimum_nights,days_to_beg_isl,days_to_end_isl,true_availability
0,0,6,2023-03-25,f,10.0,0.0,0.0,False
1,1,6,2023-03-26,f,10.0,0.0,0.0,False
2,2,6,2023-03-27,f,10.0,0.0,0.0,False
3,3,6,2023-03-28,f,10.0,0.0,0.0,False
4,4,6,2023-03-29,f,10.0,0.0,0.0,False


In [106]:
#Changing the date col to a datetime col, and creating avail month col
avail_table['date'] = pd.to_datetime(avail_table['date'])
avail_table['avail_month'] = avail_table['date'].dt.month
avail_table['avail_month'] = avail_table['avail_month'].astype(str)
print(avail_table.dtypes)
print(avail_table.head())

Unnamed: 0                    int64
listing_id                    int64
date                 datetime64[ns]
available                    object
minimum_nights              float64
days_to_beg_isl             float64
days_to_end_isl             float64
true_availability              bool
avail_month                  object
dtype: object
   Unnamed: 0  listing_id       date available  minimum_nights  \
0           0           6 2023-03-25         f            10.0   
1           1           6 2023-03-26         f            10.0   
2           2           6 2023-03-27         f            10.0   
3           3           6 2023-03-28         f            10.0   
4           4           6 2023-03-29         f            10.0   

   days_to_beg_isl  days_to_end_isl  true_availability avail_month  
0              0.0              0.0              False           3  
1              0.0              0.0              False           3  
2              0.0              0.0              False    

In [107]:
#Removing all columns which aren't listing_id, avail_month, and true_availability
select_avail = avail_table.loc[:, ['listing_id','avail_month','true_availability']]
select_avail.head()

,listing_id,avail_month,true_availability
0,6,3,False
1,6,3,False
2,6,3,False
3,6,3,False
4,6,3,False


In [108]:
#Remove duplicates from the table
sel_avail_tf = select_avail.drop_duplicates()
print("The original length of the table was",len(select_avail),"The new length of the table is",len(sel_avail_tf))

The original length of the table was 21966856 The new length of the table is 854901


In [112]:
#Combine the avail_month rows into a single col per listing id with comma separated values

#Checking "before" view
print(sel_avail_tf.loc[sel_avail_tf['listing_id'] ==29967])

#Combining rows
comb_avail = sel_avail_tf.groupby(['listing_id','true_availability'])['avail_month'].apply(','.join).reset_index()

#Checking the "after"
print(comb_avail.loc[comb_avail['listing_id'] == 29967])

       listing_id avail_month  true_availability
13870       29967           3              False
13877       29967           4              False
13907       29967           5              False
13938       29967           6              False
13942       29967           6               True
13968       29967           7              False
13999       29967           8              False
14010       29967           8               True
14030       29967           9              False
14060       29967          10              False
14091       29967          11              False
14121       29967          12              False
14152       29967           1               True
14183       29967           2               True
14212       29967           3               True
    listing_id  true_availability             avail_month
66       29967              False  3,4,5,6,7,8,9,10,11,12
67       29967               True               6,8,1,2,3


In [119]:
#Removing the unavailable listings
is_avail = comb_avail.loc[comb_avail['true_availability']==True]
is_avail.true_availability.unique()

array([ True])

In [120]:
#Saving the output
is_avail.to_csv('available_listings.csv')

### Joining the availability and review tables

In [121]:
#Loading in table with 2 columns - listing_id and avail_month
#listing_id is the primary key, avail month is a column containing a string with all the months of availability for the property 
#e.g., '1,2,3' for Jan, Feb, Mar.
avail_table = is_avail.copy()
avail_table.head()

,listing_id,true_availability,avail_month
2,109,True,"4,5,6,7,8,9,10,11,12,1,2,3"
4,2708,True,"6,7,8,9,10,11,12,1,2,3"
6,2732,True,"3,4,5,6,7,8,9,10,11,12,1,2"
8,3021,True,"3,4,5,6,7,8,9,10,11,12,1,2"
10,5728,True,"5,6,7,8,9"


In [128]:
#Match the availability table to the review information table
review_avail = pd.merge(final_reviews, avail_table, how = "left", on = "listing_id")
#review_avail = review_avail.drop('Unnamed: 0', axis = 1)
review_avail.head()

,listing_id,id,reviewer_id,reviewer_name,review_month,true_availability,avail_month
0,12899,24767,69327,Stuart,1,NaN,NaN
1,12899,29230,72846,John,3,NaN,NaN
2,12899,29806,84196,Lois,3,NaN,NaN
3,12899,32572,89114,Troy,3,NaN,NaN
4,12899,32862,100318,Cathy,4,NaN,NaN


In [130]:
#Checking that rows are updated correctly
nan_listing_list = review_avail[review_avail['avail_month'].isna()]['listing_id'].values #210227
print("There are",len(nan_listing_list),"listing ids which have null avail_month")

nan_avail_df = comb_avail.loc[comb_avail['listing_id'].isin(nan_listing_list)]
print("They only have the value",nan_avail_df.true_availability.unique(),"for availability and were removed prior to the \
join with the reviews table")

#Find Nans
print(review_avail.isnull().sum())

There are 210227 listing ids which have null avail_month
They only have the value [False] for availability and were removed prior to the join with the reviews table
listing_id                0
id                        0
reviewer_id               0
reviewer_name             0
review_month              0
true_availability    210227
avail_month          210227
dtype: int64


In [148]:
#Changing the null values in true availability and avail_month to zero
review_avail['true_availability'] = review_avail['true_availability'].fillna(False)
review_avail['avail_month'] = review_avail['avail_month'].fillna(0)

#Find Nans
print(review_avail.isnull().sum())

listing_id           0
id                   0
reviewer_id          0
reviewer_name        0
review_month         0
true_availability    0
avail_month          0
dtype: int64


In [150]:
#Add the availability in review month indicator column
review_avail['avail_ind'] = review_avail.apply(lambda review_avail: str(review_avail.review_month) in str(review_avail.avail_month), axis = 1)
review_avail

,listing_id,id,reviewer_id,reviewer_name,review_month,true_availability,avail_month,avail_ind
0,12899,24767,69327,Stuart,1,False,0,False
1,12899,29230,72846,John,3,False,0,False
2,12899,29806,84196,Lois,3,False,0,False
3,12899,32572,89114,Troy,3,False,0,False
4,12899,32862,100318,Cathy,4,False,0,False
...,...,...,...,...,...,...,...,...
2573157,806288048006012567,840008042809282936,219002106,Cheryl,3,True,"4,5,6,7,8,9,10,11,12,1,2,3",True
2573158,806288048006012567,840728285056856719,52054229,Sarah,3,True,"4,5,6,7,8,9,10,11,12,1,2,3",True
2573159,808194120211324072,822551660654988407,82164517,Jody,2,True,"5,6,7,8",False
2573160,810859442087199945,818218402306004794,415177874,Alanmichael,2,True,"3,4,5,6,7,8,9,10,11,12,1,2",True


In [151]:
#Updates avail_ind from boolean to 1/0
review_avail.loc[review_avail['avail_ind'] == True, 'avail_ind'] = 1
review_avail.loc[review_avail['avail_ind'] == False, 'avail_ind'] = 0
review_avail.loc[review_avail['listing_id']==29967]

,listing_id,id,reviewer_id,reviewer_name,review_month,true_availability,avail_month,avail_ind
1823533,29967,62788,151260,Debbie,7,True,"6,8,1,2,3",0
1823779,29967,64568,141552,Eric,7,True,"6,8,1,2,3",0
1823780,29967,67502,141591,David,7,True,"6,8,1,2,3",0
1823781,29967,70466,125982,Anders,7,True,"6,8,1,2,3",0
1823782,29967,74876,29835,Miyoko,8,True,"6,8,1,2,3",1
...,...,...,...,...,...,...,...,...
1824211,29967,699404202608288521,119169567,David,8,True,"6,8,1,2,3",1
1824212,29967,779796109146882223,85811000,Sophia,12,True,"6,8,1,2,3",0
1824213,29967,787080289800475459,15248294,Parry,12,True,"6,8,1,2,3",0
1824214,29967,791436838345282267,9251484,Allison,12,True,"6,8,1,2,3",0


In [154]:
#Verifying that everything with true availability == False has avail_ind == 0
false_avail = review_avail.loc[review_avail['true_availability']== False]
print(false_avail.avail_ind.unique())

#Verifying that everything with avail_month == 0 has avail_ind == 0
false_avail = review_avail.loc[review_avail['avail_month']== 0]
print(false_avail.avail_ind.unique())

[0]
[0]


In [155]:
#Saving the output data to csv
review_avail.to_csv('q6a_data.csv')